In [1]:
import pandas as pd
import json

with open("completos.json", "r") as f:
    dados = json.load(f)

df = pd.DataFrame(dados)
df.head()

,banco,codigoBanco,agencia,valor,linhaDigitavel,classe
0,Itau,341,0773,890.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras
1,Itau,341,0773,467.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras
2,Itau,341,0773,230.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras
3,Itau,341,0773,986.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras
4,Itau,341,0773,766.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras


In [2]:
bancos_validos = {
    "Banco do Brasil": "001",
    "Bradesco": "237",
    "Itaú": "341",
    "Caixa Econômica": "104",
    "Santander": "033"
}

agencias_validas = {
    "Banco do Brasil": ["2889", "2811", "0007", "0325", "1620"],
    "Bradesco": ["3201", "2560", "6083", "2322", "2300"],
    "Itaú": ["0364", "3174", "0814", "1594", "0773"],
    "Caixa Econômica": ["0045", "0047", "3484", "0923", "0867"],
    "Santander": ["3295", "4419", "4159", "4048", "4052"]
}


In [3]:
df['codigoBanco_bate_com_nome'] = df.apply(
    lambda row: 1 if bancos_validos.get(row['banco']) == row['codigoBanco'] else 0,
    axis=1
)

df['agencia_bate_com_banco'] = df.apply(
    lambda row: 1 if row['agencia'] in agencias_validas.get(row['banco'], []) else 0,
    axis=1
)

df['inicio_linha_bate_com_codigoBanco'] = df.apply(
    lambda row: 1 if row['linhaDigitavel'][:3] == row['codigoBanco'] else 0,
    axis=1
)

df['valor_bate_com_fim_linha'] = df.apply(
    lambda row: 1 if row['linhaDigitavel'][-10:] == row['valor'].replace('.', '').zfill(10) else 0,
    axis=1
)

df['quarto_digito_e_nove'] = df.apply(
    lambda row: 1 if row['linhaDigitavel'][3] == '9' else 0,
    axis=1
)

In [4]:
df.head()


,banco,codigoBanco,agencia,valor,linhaDigitavel,classe,codigoBanco_bate_com_nome,agencia_bate_com_banco,inicio_linha_bate_com_codigoBanco,valor_bate_com_fim_linha,quarto_digito_e_nove
0,Itau,341,0773,890.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras,0,0,1,1,1
1,Itau,341,0773,467.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras,0,0,1,1,1
2,Itau,341,0773,230.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras,0,0,1,1,1
3,Itau,341,0773,986.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras,0,0,1,1,1
4,Itau,341,0773,766.00,34190.00000 00000.000000 00000.000000 0 000000...,caracteristicasVerdadeiras,0,0,1,1,1


In [5]:
df['classe'] = df['classe'].map({
    "caracteristicasVerdadeiras": 1,
    "caracteristicasFalsas": 0
})

X = df[['codigoBanco_bate_com_nome', 'agencia_bate_com_banco', 'valor_bate_com_fim_linha', 'inicio_linha_bate_com_codigoBanco', 'quarto_digito_e_nove']]
y = df['classe']

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Acurácia: 0.95
              precision    recall  f1-score   support

           0       1.00      0.91      0.95        54
           1       0.90      1.00      0.95        46

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100



In [7]:
def prever_fraude(dado):
    banco = dado['banco']
    linha = dado['linhaDigitavel']

    dado_transformado = {
        "codigoBanco_bate_com_nome": 1 if bancos_validos.get(banco) == dado['codigoBanco'] else 0,
        "agencia_bate_com_banco": 1 if dado['agencia'] in agencias_validas.get(banco, []) else 0,
        "valor_bate_com_fim_linha": 1 if linha[-10:] == dado['valor'].replace('.', '').zfill(10) else 0,
        "inicio_linha_bate_com_codigoBanco": 1 if linha[:3] == dado['codigoBanco'] else 0,
        "quarto_digito_e_nove": 1 if linha[3] == '9' else 0
    }

    df_input = pd.DataFrame([dado_transformado])
    pred = model.predict(df_input)[0]
    return "Verdadeiro" if pred == 1 else "Falso"


In [20]:
prever_fraude({
    "banco": "Banco do Brasil",
    "agencia": "2889",
    "codigoBanco": "001",
    "valor": "720.00",
    "linhaDigitavel": "00190.00000 00000.000000 00000.000000 0 00000000072000"
})


'Verdadeiro'